In [1]:
import jax
import jax.numpy as jnp
import numpy as np

from jax import grad, jit, vmap, pmap

from jax import random
import matplotlib.pyplot as plt
from copy import deepcopy
from typing import Tuple, NamedTuple
import functools

#### We built and trained an MLP using JAX in the last notebook but we cannot similarly build other NN Libraries or Layers such as NN.Linear without more knowledge

## Custom PyTrees

In [2]:
# Imagine a linear layer or a convolution layer
class MyContainer:
    """A named Container"""

    def __init__(self, name:str, a:int, b:int, c:int):
        self.name = name
        self.a = a
        self.b = b
        self.c = c   

In [3]:
example_pytree = [MyContainer('Alice', 1, 2, 3), MyContainer("Bob", 4, 5, 6)]
# We would think this Pytree will have 8 leaves
# Nope!

In [5]:
leaves = jax.tree.leaves(example_pytree)
print(f"{repr(example_pytree):<45}\n has {len(leaves)} leaves: \n{leaves}")

[<__main__.MyContainer object at 0x0000019FDC21F230>, <__main__.MyContainer object at 0x0000019FAACEE0D0>]
 has 2 leaves: 
[<__main__.MyContainer object at 0x0000019FDC21F230>, <__main__.MyContainer object at 0x0000019FAACEE0D0>]


- Only 2 leaves!
- In this case, we cannot do manipulations on each value (like use tree.map)
- Image if this was a Linear Layer, we wont be able to do any Stochastic GD

In [6]:
print(jax.tree.map(lambda x: x+1, example_pytree))
# This will not work (would be nice if it did)

TypeError: unsupported operand type(s) for +: 'MyContainer' and 'int'

#### To get it to work, we need define two functions:
- flatten
- unflatten